In [1]:
from data import load_data, GS_Dataset, make_gs_dataset, GS_Results
from model import Narx, DEVICE
from util_fun import train_narx_simval, narx_sim_nrms
import torch
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split



In [2]:
n_a=15
n_b=15
n_layers=5
n_nodes=50
n_epochs=15000

In [3]:
model=Narx(n_a+n_b,n_nodes, n_layers).to(DEVICE)

In [4]:
x_data, y_data= load_data()
data = make_gs_dataset(x_data, y_data, n_a, n_b, DEVICE)

In [5]:
results:GS_Results = train_narx_simval(model, n_a, n_b, data, n_epochs=n_epochs)

Checkpoint at epoch 1:  

current best pred NRMS: 1.000007530937103, previous best pred NRMS: inf 

current best sim NRMS: 1.0001159935167798, previous best sim NRMS: inf 

Checkpoint at epoch 601:  

current best pred NRMS: 0.3855354382230041, previous best pred NRMS: 1.000007530937103 

current sim NRMS: 1.259683803891892, current best sim NRMS: 1.0001159935167798 

Checkpoint at epoch 1201:  

current best pred NRMS: 0.358051244226336, previous best pred NRMS: 0.3855354382230041 

current sim NRMS: 1.2379272281691092, current best sim NRMS: 1.0001159935167798 

Checkpoint at epoch 1801:  

current best pred NRMS: 0.3372578725807001, previous best pred NRMS: 0.358051244226336 

current sim NRMS: 1.3087576134231065, current best sim NRMS: 1.0001159935167798 

Checkpoint at epoch 2401:  



KeyboardInterrupt: 

In [ ]:
torch.save(results.best_model.state_dict(), 'narx15K_na15_nb15_nlay5_nnode50_pred')